## 날짜설정

In [9]:
import datetime

today = datetime.date.today()

# 특정 날짜 기준으로 설정하고 싶을때는 아래 주석 해제하여 사용
#today = datetime.date(2022,9,1)

day_1 = today - datetime.timedelta(1)
start_day = datetime.date(year = day_1.year, month=day_1.month, day=1)
day_1_yearmonth = day_1.strftime('%Y%m')

## 디렉토리 설정

In [10]:
# 데이터컨설팅 폴더 동기화 필요
# 자신의 드롭박스 경로에 맞게 수정
dropbox_dir = 'C:/Users/MADUP/Dropbox (주식회사매드업)/광고사업부/데이터컨설팅/Tableau'

raw_dir = dropbox_dir + '/RD'   # 리포트 RD 업데이트 하는 경로
                                # raw_dir에는 result_name_yyyymm.csv 형태로 저장되어 있어야 합니다.
result_dir = dropbox_dir + '/result' # 태블로용 RD 저장 폴더

token_dir = dropbox_dir + '/token'
asset_dir = dropbox_dir + '/asset'

## 광고주 정보

In [11]:
class info():
    account_name = '핀다' # 광고주 정보
    result_name = 'finda'# 파일명 정보

## 스프레드 시트 설정

In [12]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import time

class spread_sheet():
    def spread_document_read(url):
        scope = ['https://spreadsheets.google.com/feeds',
                 'https://www.googleapis.com/auth/drive']
        json_file = token_dir + '/madup-355605-cd37b0ac201f.json'
        credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file, scope)
        gc = gspread.authorize(credentials)
        read = gc.open_by_url(url)
        print('스프레드시트 읽기 완료')
        return read

    def spread_sheet(doc, sheet_name, col_num=0, row_num=0):
        while True :
            try :
                data_sheet = doc.worksheet(sheet_name)
                data_sheet_read = data_sheet.get_all_values()
                result = pd.DataFrame(data_sheet_read, columns=data_sheet_read[row_num]).iloc[row_num+1:, col_num:]
            except :
                print('API 오류로 15초 후 다시 시도 합니다.')
                time.sleep(15)
                continue
            break
        return result

## 코드 실행

In [19]:
import pandas as pd

# 태블로 대시보드 컬럼 설정 불러오기
doc = spread_sheet.spread_document_read('https://docs.google.com/spreadsheets/d/1efQrJpglhz0K9wKkl2QZ5VLrMu_hIUUnPG_uTSeI1Hk/edit#gid=2008544363')
sheet_data = spread_sheet.spread_sheet(doc, info.account_name)

sheet_data_column = sheet_data.loc[sheet_data['원본 리포트 기준']!= '입력 금지', ['원본 리포트 기준', '태블로 RD']]
sheet_data_column = sheet_data_column.loc[sheet_data_column['원본 리포트 기준']!='']

# 컬럼명 기준 딕셔너리 생성
column_dict = dict(zip(sheet_data_column['원본 리포트 기준'], sheet_data_column['태블로 RD']))
use_cols = column_dict.keys()

## 광고주 리포트 데이터 가져오기
raw_data = pd.read_csv(raw_dir + f'/{info.account_name}/{info.result_name}_{day_1_yearmonth}.csv',encoding='utf-8-sig',usecols= use_cols)

raw_data = raw_data.rename(columns = column_dict)
raw_data = raw_data.loc[pd.to_datetime(raw_data['날짜']).dt.month == day_1.month]
raw_data.columns
# string to numeric
value_columns = ['노출', '클릭', '비용', '조회','대출실행','설치','가입','한도조회','마이데이터',
                 '재설치','재유입','m.설치', 'm.가입', 'm.한도조회', 'm.대출실행']
columns = raw_data.columns

for col in value_columns:
    if col in columns:
        if raw_data[col].dtypes == 'object':
            raw_data[col] = raw_data[col].str.strip()
            raw_data[col] = raw_data[col].str.replace('-', '0')
            raw_data[col] = raw_data[col].str.replace(',', '')
            raw_data[col] = pd.to_numeric(raw_data[col])

# 매체 정보 컬럼 추가
media_category = sheet_data[sheet_data.columns[6:]]

for category in media_category.columns :
    medias = list(media_category[category].unique())
    raw_data.loc[raw_data['매체'].isin(medias), '광고 유형'] = category


# 캠페인 정보 관리 시트 불러오기
campaign_doc = spread_sheet.spread_document_read('https://docs.google.com/spreadsheets/d/1w23AdEYLf4OJDlTElf2vwbVPttctSx-2nB51ZHQDEJQ/edit#gid=0')
campaign_sheet = spread_sheet.spread_sheet(campaign_doc, '캠페인 정보')
campaign_sheet = campaign_sheet.loc[campaign_sheet['광고주'] == '핀다']
#campaign_sheet = campaign_sheet.loc[campaign_sheet['매체'].isin(['페이스북', '카카오', '구글'])]
campaign_sheet = campaign_sheet[['광고 상품', '캠페인', '최적화 엔진', '캠페인 구분', '오디언스', 'OS', 'KPI', '캠페인 라벨']]
campaign_sheet = campaign_sheet.drop_duplicates(['캠페인'], keep='last')

### 리포트 RD 캠페인 정보 merge
raw_data = raw_data.merge(campaign_sheet, on=['캠페인'], how='left')

# creative report

creative_raw = raw_data.copy()

# 매체별 필터 기준
facebook_list = list(sheet_data.loc[sheet_data['페이스북 필터'].str.len() > 0, '페이스북 필터'])
kakao_list = list(sheet_data.loc[sheet_data['카카오 필터'].str.len() > 0, '카카오 필터'])
google_list = list(sheet_data.loc[sheet_data['구글 필터'].str.len() > 0, '구글 필터'])

# 예외처리 정보 불러오기
exception_sheet = spread_sheet.spread_sheet(doc, '예외처리')
exception_sheet = exception_sheet.loc[(exception_sheet['체크박스']=='TRUE')&(exception_sheet['광고주']==info.account_name)]
exception_sheet.index = range(0, len(exception_sheet))

for i in exception_sheet.index:
    func = exception_sheet.iloc[i]['방식']
    col = exception_sheet.iloc[i]['구분']
    media = exception_sheet.iloc[i]['매체']
    name = exception_sheet.iloc[i]['이름']
    alt = exception_sheet.iloc[i]['대체 값']

    if func == 'EXCLUDE':
        creative_raw = creative_raw.loc[~((creative_raw['매체']==media) & (creative_raw[col]==name))]
    elif func == 'REPLACE' :
        creative_raw.loc[((creative_raw['매체'] == media) & (creative_raw[col] == name)), name] = alt

#creative_raw = creative_raw.loc[creative_raw['매체'].isin(['페이스북', '카카오', '구글'])]
creative_raw.loc[creative_raw['매체'].isin(google_list), '소재'] = creative_raw['광고그룹']


# 애셋 정보 불러오기
account_sheet = spread_sheet.spread_sheet(campaign_doc, 'Owner 정보')
account_sheet = account_sheet.rename(columns = {'Owner ID' : 'owner_id'})
account_sheet = account_sheet.drop_duplicates('owner_id')
asset_data = pd.read_csv(asset_dir + f'/total_asset_data_{day_1_yearmonth}.csv')
asset_data_merge = asset_data.merge(account_sheet, on ='owner_id', how='left')
asset_data_merge = asset_data_merge.loc[asset_data_merge['광고주']=='핀다']
del asset_data_merge['owner_id'], asset_data_merge['광고주'], asset_data_merge['매체']

creative_merge_final = creative_raw.merge(asset_data_merge, on = ['캠페인', '광고그룹', '소재'], how = 'left')

# 수동 예외처리
# creative_merge_final.loc[creative_merge_final['소재'].str.contains(pat = '@AOS'), 'OS'] = 'AOS'
# creative_merge_final.loc[creative_merge_final['소재'].str.contains(pat = '@IOS'), 'OS'] = 'iOS'
creative_merge_final.loc[creative_merge_final['캠페인 구분'] == 'Retargeting', '재한도조회'] = creative_merge_final['한도조회']
creative_merge_final.loc[creative_merge_final['캠페인 구분'] == 'Retargeting', '한도조회'] = 0

## 추가 데이터 불러오기
rd_doc =  spread_sheet.spread_document_read('https://docs.google.com/spreadsheets/d/1Sj7AFdWkM87yn8RwMZLSJ-WHav0WqX4xsBrm0iivSfY/edit#gid=0')
kpi_data = spread_sheet.spread_sheet(rd_doc, 'KPI')
kpi_data = kpi_data.rename(columns = {'노출' : '노출 KPI',
                                      '클릭' : '클릭 KPI',
                                      '비용' : '비용 KPI',
                                      '설치' : '설치 KPI',
                                      '재설치' : '재설치 KPI',
                                      '가입' : '가입 KPI',
                                      '한도조회' : '한도조회 KPI',
                                      '재한도조회' : '재한도조회 KPI',
                                      '대출실행' : '대출실행 KPI',
                                      '마이데이터' : '마이데이터 KPI'})
kpi_data = kpi_data.loc[(pd.to_datetime(kpi_data['날짜']).dt.date >= start_day) & (pd.to_datetime(kpi_data['날짜']).dt.date <= day_1)]
organic_data = spread_sheet.spread_sheet(rd_doc, '오가닉')
organic_data = organic_data.loc[(pd.to_datetime(organic_data['날짜']).dt.date >= start_day) & (pd.to_datetime(organic_data['날짜']).dt.date <= day_1)]
au_data = spread_sheet.spread_sheet(rd_doc, 'MAU')
au_data = au_data.loc[(pd.to_datetime(au_data['날짜']).dt.date >= start_day) & (pd.to_datetime(au_data['날짜']).dt.date <= day_1)]


creative_merge_final = pd.concat([creative_merge_final, kpi_data, organic_data, au_data], sort=False, ignore_index=True)

value_columns = value_columns + ['노출 KPI', '클릭 KPI', '비용 KPI', '설치 KPI', '재설치 KPI', '가입 KPI', '한도조회 KPI',
                 '재한도조회 KPI',  '대출실행 KPI', 'DAU KPI','마이데이터 KPI','오가닉 설치', 'DAU', '재한도조회']

for val in value_columns :
    creative_merge_final[val] = creative_merge_final[val].fillna(0)

creative_merge_final.to_csv(result_dir + f'/{info.account_name}/tableau_creative_rd_{info.result_name}_{day_1_yearmonth}.csv', index=False, encoding='utf-8-sig')

스프레드시트 읽기 완료
스프레드시트 읽기 완료


In [6]:
creative_merge_final['한도조회'].sum()

77094.0